In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
# ========== Dataset ==========
class SteeringDataset(Dataset):
    def __init__(self, images, turn_signals, angles, transform=None):
        self.images = images
        self.turn_signals = turn_signals
        self.labels = self.classify_angles(angles)
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        signal = self.turn_signals[idx] + 1  # Map -1,0,1 to 0,1,2
        label = self.labels[idx]

        if self.transform:
            img = self.transform(img)  # Already returns a FloatTensor

        return (
            img,  # already a tensor from transform
            torch.tensor(signal, dtype=torch.long),
            torch.tensor(label, dtype=torch.long)
        )

    @staticmethod
    def classify_angles(angles):
        # Bin steering angles into 15 classes
        bins = [-1.01, -0.90, -0.70, -0.30, -0.20, -0.10, -0.05, -0.03, 0.03, 0.05, 0.10, 0.20, 0.30, 0.70, 0.90, 1.01]
        return np.digitize(angles, bins) - 1  # Result: labels 0 to 6

In [3]:
# ========== Model ==========
class SteeringClassifier(nn.Module):
    def __init__(self, num_classes=15):
        super(SteeringClassifier, self).__init__()
        resnet = models.resnet18(pretrained=True)
        resnet.fc = nn.Identity()
        self.cnn = resnet
        self.turn_embed = nn.Embedding(3, 16)  # 3 turn signals: -1, 0, 1 -> 0, 1, 2
        self.fc = nn.Sequential(
            nn.Linear(512 + 16, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )
        
    def forward(self, image, turn_signal):
        x_img = self.cnn(image)
        # x_signal = self.turn_embed(turn_signal)
        x_signal = self.turn_embed(turn_signal.to(self.turn_embed.weight.device))
        x = torch.cat((x_img, x_signal), dim=1)
        return self.fc(x)


In [4]:
def train_model(model, train_loader, val_loader, criterion, optimizer, device,
                num_epochs=10, save_path="../Models/steering_checkpoint.pth", start_epoch=0):
    model = model.to(device)
    train_loss_history, val_loss_history = [], []
    best_val_loss = float('inf')  # Initialize with infinity

    # Try to load existing checkpoint to get best_val_loss if resuming
    if os.path.exists(save_path):
        checkpoint = torch.load(save_path)
        if 'val_loss' in checkpoint:
            best_val_loss = checkpoint['val_loss']

    for epoch in range(start_epoch, num_epochs):
        # ========== Training ==========
        model.train()
        total_loss = 0
        for imgs, signals, labels in train_loader:
            imgs, signals, labels = imgs.to(device), signals.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(imgs, signals)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        train_loss = total_loss / len(train_loader)
        train_loss_history.append(train_loss)

        # ========== Validation ==========
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for imgs, signals, labels in val_loader:
                imgs, signals, labels = imgs.to(device), signals.to(device), labels.to(device)
                outputs = model(imgs, signals)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        val_loss = val_loss / len(val_loader)
        val_loss_history.append(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # ========== Save best checkpoint ==========
        if val_loss < best_val_loss:
            print(f"✔️  Saving new best model at epoch {epoch+1} (Val Loss: {val_loss:.4f})")
            best_val_loss = val_loss
            torch.save({
                'epoch': epoch + 1,
                'model_state': model.state_dict(),
                'optimizer_state': optimizer.state_dict(),
                'val_loss': val_loss
            }, save_path)

    return train_loss_history, val_loss_history


In [5]:
def load_checkpoint(model, optimizer, checkpoint_path, device):
    checkpoint = torch.load(checkpoint_path, map_location=device)  # map checkpoint tensors to correct device
    model.load_state_dict(checkpoint['model_state'])
    optimizer.load_state_dict(checkpoint['optimizer_state'])

    # Ensure optimizer tensors are moved to the right device
    for state in optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.to(device)

    model = model.to(device)  # Just to be safe
    start_epoch = checkpoint.get('epoch', 0)
    val_loss = checkpoint.get('val_loss', None)

    if val_loss is not None:
        print(f"✅ Loaded checkpoint from epoch {start_epoch} (val_loss = {val_loss:.4f})")
    else:
        print(f"✅ Loaded checkpoint from epoch {start_epoch} (val_loss not recorded)")

    return model, optimizer, start_epoch


In [6]:
# ========== Plot ==========
def plot_training_curves(train_loss, val_loss):
    plt.figure()
    plt.plot(train_loss, label='Train Loss')
    plt.plot(val_loss, label='Val Loss')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training vs Validation Loss")
    plt.legend()
    plt.grid(True)
    plt.show()

In [7]:
# ========== Save Final ==========
def save_final_model(model, path="../Models/steering_model_final.pth"):
    torch.save(model.state_dict(), path)

In [ ]:
# ========== Main ==========
if __name__ == "__main__":
    # Load data
    images = np.load("../output/all_images.npy")               # shape (N, 3, H, W) or (N, H, W, 3)
    turn_signals = np.load("../output/all_turn_signals.npy")   # values: -1, 0, 1
    angles = np.load("../output/all_angles.npy")               # values: -1.0 to 1.0

    # Fix image shape if needed
    if images.ndim == 4 and images.shape[1] == 3:
        images = np.transpose(images, (0, 2, 3, 1))  # to HWC

    # Normalize for uint8 display (optional)
    if images.max() <= 1.0:
        images = (images * 255).astype(np.uint8)
    else:
        images = images.astype(np.uint8)

    # Transform
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),  
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])


    # # Split data
    # X_train, X_val, s_train, s_val, a_train, a_val = train_test_split(
    #     images, turn_signals, angles, test_size=0.2, random_state=42
    # )

    split_idx = int(0.8 * len(images))
    X_train, X_val = images[:split_idx], images[split_idx:]
    s_train, s_val = turn_signals[:split_idx], turn_signals[split_idx:]
    a_train, a_val = angles[:split_idx], angles[split_idx:]

    # Datasets and Loaders
    train_dataset = SteeringDataset(X_train, s_train, a_train, transform)
    val_dataset = SteeringDataset(X_val, s_val, a_val, transform)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    # Model, Optimizer, Loss
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SteeringClassifier()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-6, weight_decay=1e-5)

    # === Try loading checkpoint if it exists ===
    checkpoint_path = "../Models/quaternary_model_checkpoint.pth"
    start_epoch = 0
    if os.path.exists(checkpoint_path):
        model, optimizer, start_epoch = load_checkpoint(model, optimizer, checkpoint_path, device)
    else:
        model = model.to(device)  # If no checkpoint, still move model to device
        print("No checkpoint found, starting from scratch.")
    
    # Train
    train_loss, val_loss = train_model(
        model, train_loader, val_loader,
        criterion, optimizer, device,
        num_epochs=12,  # Total number of epochs you want
        save_path=checkpoint_path,
        start_epoch=start_epoch  # Resume from this
    )
    # 
    # Plot and Save
    plot_training_curves(train_loss, val_loss)
    save_final_model(model, "../Models/quaternary_model_final.pth")

✅ Loaded checkpoint from epoch 2 (val_loss = 0.5545)
Epoch 3/12 - Train Loss: 0.2422, Val Loss: 0.5369
✔️  Saving new best model at epoch 3 (Val Loss: 0.5369)
Epoch 4/12 - Train Loss: 0.2057, Val Loss: 0.5619
Epoch 5/12 - Train Loss: 0.1745, Val Loss: 0.7759
Epoch 6/12 - Train Loss: 0.1368, Val Loss: 0.6311
Epoch 7/12 - Train Loss: 0.1171, Val Loss: 0.7272
Epoch 8/12 - Train Loss: 0.0901, Val Loss: 0.6790
Epoch 9/12 - Train Loss: 0.0677, Val Loss: 0.7555
Epoch 10/12 - Train Loss: 0.0591, Val Loss: 0.7890


KeyboardInterrupt: 